In [6]:
import rasterio
from shapely.geometry import mapping
import os
import rasterio.mask
from rasterio.mask import mask
import rasterio.features
import geopandas as gpd
import pandas as pd
from shapely.geometry import Polygon
import matplotlib.pyplot as plt
import numpy as np
from scipy.ndimage import uniform_filter
import elevation
import tempfile
import seaborn as sns
import osmnx as ox
import pyrosm
from src.utils import DATA_DIR, KILOMETERS_OFFSHORE

In [7]:
plt.style.use('dark_background')

In [8]:
brazil = gpd.read_file(f'{DATA_DIR}/brazil.geojson')
offshore = gpd.read_file(f'{DATA_DIR}/offshore.geojson')

## Meterology

In [5]:

def get_power_density(geometries):
    with rasterio.open(f'{DATA_DIR}/meterology/gwa3_250_powerdensity_100m.tif') as src:
        power_density, out_transform = rasterio.mask.mask(src, geometries, crop=True)
        power_density = power_density[0]
    return power_density, src.meta

def get_wind_speed(geometries):
    with rasterio.open(f'{DATA_DIR}/meterology/gwa3_250_windspeed_100m.tif') as src:
        wind_speed, out_transform = rasterio.mask.mask(src, geometries, crop=True)
        wind_speed = wind_speed[0]
    return wind_speed


In [11]:
geometries = [mapping(polygon) for polygon in brazil.union(offshore).geometry]
wind_speed = get_wind_speed(geometries)
power_density = get_power_density(geometries)

## Demography

In [12]:
def get_population_density():
    population_density = pd.read_csv(f'{DATA_DIR}/demography/bra_pd_2020_1km_ASCII_XYZ.csv')
    population_density = gpd.GeoDataFrame(population_density['Z'],
        geometry=gpd.points_from_xy(population_density['X'], population_density['Y']))
    return population_density

In [13]:
population_density = get_population_density()

## Topography

In [14]:
def get_topography():
    with rasterio.open(f'{DATA_DIR}/topography/brazil_dem.tif') as src:
        topography = src.read(1)
    return topography

def get_slope():
    with rasterio.open(f'{DATA_DIR}/topography/slope.tif') as src:
        slope = src.read(1)
    return slope

def get_roughness():
    with rasterio.open(f'{DATA_DIR}/topography/roughness.tif') as src:
        roughness = src.read(1)
    return roughness

In [15]:
topography = get_topography()
slope      = get_slope()
roughness  = get_roughness()

## Biodiversity

## Infrastructure

In [2]:
osm = pyrosm.OSM(f'{DATA_DIR}/infrastructure/brazil.pbf')
power = osm.get_data_by_custom_criteria(custom_filter={"power": ["line"]})

In [ ]:
# save power lines
power.to_file(f'{DATA_DIR}/infrastructure/power.geojson', driver='GeoJSON')

## Analysis

In [ ]:
# this metereology mask is rater based and has the crs of the raster
metereology_mask = (wind_speed > 8) & (wind_speed < 12) & (power_density > 200)

In [ ]:

population_mask = population_density['Z'] < 1

In [21]:
wind_speed

array([[-999., -999., -999., ..., -999., -999., -999.],
       [-999., -999., -999., ..., -999., -999., -999.],
       [-999., -999., -999., ..., -999., -999., -999.],
       ...,
       [-999., -999., -999., ..., -999., -999., -999.],
       [-999., -999., -999., ..., -999., -999., -999.],
       [-999., -999., -999., ..., -999., -999., -999.]], dtype=float32)